# <center> Final Project - Being Invisible </center>

### Import the Required Libraries:

In [15]:
import numpy as np
import cv2
import time

### Processing the video:

- In cv2.VideoCapture(0), you can also use a previously recored video instead of '0', the default camera.
- However note that the first 15 seconds of the video will be considered as the background of the image

In [16]:
# Initialize a video capture object
video = cv2.VideoCapture(0)

for i in range(5):
    time.sleep(1)
    _, frame = video.read()
    
background = frame
    
# Display the background, with window titled 'Your Picture'
cv2.imshow('Your Background', background)

# Capturing Live frame:

while(True):
    #Read an image frome
    _, img = video.read()
    
    # Convert to HSV Colorspace
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    
    # Range for blue (in HSV format)
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])
    
    # Create a mask for the specified range
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Applying some transforms to the mask for better results:
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) # This is just dilation followed by erosion
    
    # Creating an inverted mask which when 'AND'ed with the original image will remove the mask
    inv_mask = cv2.bitwise_not(mask)
    
    # Removing the mask from the frame
    img1 = cv2.bitwise_and(img, img, inv_mask)
    
    # Creating dynamic background showing background for pixels which are segmented out by the inv_mask
    back_dynamic = cv2.bitwise_and(background, background, mask = mask)
    
    # Display the image frame, with window titled 'Your Picture'
    #Generating the final output
    final_output = cv2.add(img1,back_dynamic)
    cv2.imshow("Invisible",final_output)

    # Wait for 1 milliseconds or until a key is pressed
    key = cv2.waitKey(1) & 0xFF 
    
    # Exit the loop when q is pressed
    if key == ord('q'):
        break

cv2.destroyAllWindows()
video.release()

## Priniciple:

- Firstly, the video capture objects waits for 5 seconds, after which the last frame captured is considered as the background.


- This background is what will be displayed when an object with colour in the range of the mask appears in the mask.

## Note:

1) We convert to HSV colour format because it can easily be used with a variety of brightness levels of the same colour, whereas same cannot be said about the RGB format.

2) If you want to use 'Red' coloured mask, there will be two masks which need to be added together. This is because in OpenCV's HSV format 'Red' colour is represented between Hue values: 0 - 10 & 170 - 180.